In [1]:
import numpy as np
import pandas as pd
import ipympl
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
from echopop import Survey
from echopop.spatial.variogram import get_variogram_arguments
import copy
from echopop.spatial.transect import edit_transect_columns
from echopop.spatial.projection import transform_geometry
from echopop.analysis import variogram_analysis
from echopop.spatial.mesh import griddify_lag_distances
from echopop.spatial.variogram import empirical_variogram
from IPython.display import display, clear_output
from echopop.spatial.variogram import variogram, optimize_variogram

In [2]:

survey = Survey(init_config_path="C:/Users/Brandyn/Documents/GitHub/echopop/config_files/initialization_config.yml" ,
                survey_year_config_path="C:/Users/Brandyn/Documents/GitHub/echopop/config_files/survey_year_2019_config.yml")
survey.transect_analysis()
self = survey
settings_dict = {"variable": "biomass_density",
                 "stratum_name": "stratum_num",
                 "kriging_parameters": {
                    "longitude_reference": -124.78338,
                    "latitude_offset": 45.0,
                    "longitude_offset": -124.78338,
                 }}
# Prepare the transect data
# ---- Create a copy of the transect dictionary
transect_input = copy.deepcopy(self.analysis["transect"])
# ---- Edit the transect data
transect_data = edit_transect_columns(transect_input, settings_dict)
isobath_df = self.input["statistics"]["kriging"]["isobath_200m_df"]
transect_data, _, _ = transform_geometry(transect_data, isobath_df, settings_dict)

# Get the stratum name
stratum_name = self.analysis["settings"]["transect"]["stratum_name"]

# Get standardization config for kriging 
standardization_parameters = self.input["statistics"]["kriging"]["model_config"]
# ---- Get isobath data
isobath_df = self.input["statistics"]["kriging"]["isobath_200m_df"]

# Get variogram parameters
variogram_parameters = self.input["statistics"]["variogram"]["model_config"].copy()

# Generate settings dictionary
settings_dict = {
    "stratum_name": stratum_name,
    "verbose": False,
    "kriging_parameters": {
        "longitude_reference": standardization_parameters["longitude_reference"],
        "longitude_offset": standardization_parameters["longitude_offset"],
        "latitude_offset": standardization_parameters["latitude_offset"],
    },
}


# # Create a copy of the existing variogram settings
# variogram_parameters = self.input["statistics"]["variogram"]["model_config"].copy()
# n_lags = 30
# lag_resolution = variogram_parameters["lag_resolution"]
# # ---- Number of lags
# variogram_parameters["n_lags"] = 30
# # ---- Azimuth range
# variogram_parameters["azimuth_range"] = 360
# # ---- Force lag-0
# variogram_parameters["force_lag_zero"] = True
# # Compute the lag distances
# distance_lags = np.arange(1, n_lags) * lag_resolution
# # ---- Add to the `variogram_parameters` dictionary
# variogram_parameters["distance_lags"] = distance_lags
# # ---- Update the max range parameter, if necessary
# max_range = lag_resolution * n_lags
# lags, gamma_h, lag_counts, lag_covariance = empirical_variogram(
#     transect_data, variogram_parameters, {"variable": "biomass_density"}
# )


    --------------------------------
    TRANSECT RESULTS
    --------------------------------
    | Variable: Biomass (kmt)
    | Age-1 fish excluded: True
    | Stratum definition: KS
    --------------------------------
    GENERAL RESULTS
    --------------------------------
    Total biomass: 1651.1 kmt
        Age-1: 7.9 kmt
        Age-2+: 1643.2 kmt
    Total female biomass: 832.2 kmt
        Age-1: 4.0 kmt
        Age-2+: 828.2 kmt
    Total male biomass: 818.5 kmt
        Age-1: 3.9 kmt
        Age-2+: 814.6 kmt
    Total unsexed biomass: 0.4 kmt
    Total mixed biomass: 36.8 kmt
    --------------------------------


In [6]:
VARIOGRAM_MODEL_PARAMETER_MAP = {
    "Bessel-exponential": ["bessel", "exponential"],
    "Bessel": "bessel",
    "Exponential": "exponential",
    "Gaussian": "gaussian",
    "Linear": "linear",
    "Sinc": "sinc",
    "Spherical": "spherical",
    "Bessel-Gaussian": ["bessel", "gaussian"],
    "Cosine-exponential": ["cosine", "exponential"],
    "Cosine-Gaussian": ["cosine", "gaussian"],
    "Exponential-linear": ["exponential", "linear"],
    "Gaussian-linear": ["gaussian", "linear"]
}

# Variogram model parameter map
THEORETICAL_VARIOGRAM_PARAMETER_MAP = {
    "nugget": dict(name="Nugget", widget="entry", step=0.01, string_format="0.3f"),
    "sill": dict(name="Sill", widget="entry", step=0.01, string_format="0.3f"),
    "correlation_range": dict(name="Correlation range", widget="entry", step=0.0001, string_format="0.4f"),
    "decay_power": dict(name="Decay power", widget="entry", step=0.05, string_format="0.2f"),
    "hole_effect_range": dict(name="Hole effect range", widget="entry", step=0.0001, string_format="0.3f"),
    "enhance_semivariance": dict(name="Enhance semivariance", widget="checkbox", step=np.nan)
}

# Empirical variogram general settings map
EMPIRICAL_VARIOGRAM_PARAMETER_MAP = {
    "n_lags": 30,
    "lag_resolution": 0.002,
    "azimuth_range": 360.0
}

# Optimization
OPTIMIZATION_PARAMETER_MAP = {
    "Maximum function evaluations": dict(name="max_fun_evaluations", widget="entry_int", value=500, step=1),
    "Cost function tolerance": dict(name="cost_fun_tolerance", widget="entry_float", value=1e-6, step=1e-7),
    "Solution tolerance": dict(name="solution_tolerance", widget="entry_float", value=1e-6, step=1e-5),
    "Gradient tolerance": dict(name="gradient_tolerance", widget="entry_float", value=1e-4, step=1e-5),
    "Finite differences step size": dict(name="finite_step_size", widget="entry_float", value=1e-8, step=1e-9),
    "Trust Region solver method": dict(name="trust_region_solver", widget="entry_str", value="exact", step=["exact", "base"]),
    "Characteristic x-scale": dict(name="x_scale", widget="entry_hybrid", value="jacobian", value_float=1.0, step=0.1),
    "Jacobian approximation method": dict(name="jacobian_approx", widget="entry_str", value="central", step=["central", "forward"])
}

style = {'description_width': '150px'}
layout = {'width': '400px'}

In [16]:
import warnings
from echopop.spatial.variogram import create_optimization_options, validate_variogram_parameters
from matplotlib.patches import Patch
warnings.filterwarnings("ignore", category=DeprecationWarning)

def plot_empirical_variogram(lags: np.ndarray, gamma_h: np.ndarray, lag_counts: np.ndarray, patch=None):
    # Generate point-size scaling lambda function
    size_scale = lambda x: (((x - x.min()) / float(x.max() - x.min()) + 1) * 10)**2

    # Create the plot
    fig, ax = plt.subplots(figsize=(8, 5)) # Figure and axis initialization
    scatter = ax.scatter(lags, gamma_h, s=size_scale(lag_counts), edgecolors="black",
                         label="Empirical variogram [γ(h)]") # scatter plot
    if patch is not None:
        if not isinstance(patch, list):
            ax.legend(handles=[scatter, patch], loc="lower right")
        else:
            ax.legend(handles=[scatter, patch[0], patch[1]], loc="lower right")
    else:
        ax.legend(loc="lower right")
        
    ax.set_xlabel("Lag distance [h]") # x-axis label
    ax.set_ylabel("Semivariance [γ]") # y-axis label
    ax.grid(True) # create axis grid lines
    ax.set_axisbelow(True) # set axis grid lines below points
    plt.tight_layout() # adjust padding around plotting area

    # Generate list of annotation points
    annotation_points = ax.annotate("", xy=(0, 0), xytext=(-50, -50), textcoords="offset points",
                                    bbox=dict(boxstyle="round", fc="white", ec="black", alpha=1.0),
                                    arrowprops=dict(arrowstyle="->", color="black", lw=1, 
                                                    linestyle="-"))
    # ---- Set values to be invisible 
    annotation_points.set_visible(False)

    # Helper function for revealing invisible annotations
    def reveal_annotation(ind):
        position = scatter.get_offsets()[ind["ind"][0]] # get scatter point positions
        annotation_points.xy = position # set the annotation coordinates
        # ---- Create text label
        text = f"h={position[0]:.2f}\nγ={position[1]:.2f}\nCount={lag_counts[ind['ind'][0]]}" 
        annotation_points.set_text(text) # assign text to annotation point
        annotation_points.get_bbox_patch().set_alpha(1.0) # set alpha transparency for text box

    # Helper function for target annotation
    def on_hover(event):
        vis = annotation_points.get_visible() # detect the visible annotation point
        if event.inaxes == ax:
            cont, ind = scatter.contains(event)
            # ---- If the annotation intersects with the cursor hover point
            if cont: 
                reveal_annotation(ind)
                annotation_points.set_visible(True)          
                # ---- Draw
                fig.canvas.draw_idle()
            else:
                if vis: # points that aren't 'highlighted' by the cursor
                    annotation_points.set_visible(False)
                    # ---- Draw
                    fig.canvas.draw_idle()

    # Create event connection
    fig.canvas.mpl_connect("motion_notify_event", on_hover)

    # Clean up the canvas
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False

    # Return figure and axis
    return fig, ax

def empirical_variogram_widget():

    # Create general settings value entry widgets
    # ---- Number of lags
    n_lags_entry = widgets.IntText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["n_lags"],
                                   description="Number of lags",
                                   step=1, style=style, layout=layout)
    # ---- Lag (increment) resolution
    lag_resolution_entry =  widgets.FloatText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["lag_resolution"], 
                                              description='Lag resolution', 
                                              step=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["lag_resolution"]/2, 
                                              style=style, layout=layout)
    # ---- Azimuth range 
    azimuth_range_entry = widgets.FloatText(value=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["azimuth_range"], 
                                            description='Azimuth Range', 
                                            step=EMPIRICAL_VARIOGRAM_PARAMETER_MAP["azimuth_range"]/360, 
                                            style=style, layout=layout)
    
    # Add variable dropdown widget
    empirical_variogram_dropdown = widgets.Dropdown(
        options=list(["Biomass (biomass density)", "Abundance (number density)"]),
        value="Biomass (biomass density)",
        description="Transect variable",
        disabled=False,
        style = {"description_width": "150px"}, 
        layout = {"width": "400px"}
    )  

    # Add empirical variogram button
    compute_empirical_variogram_button = widgets.Button(description="Compute Empirical Variogram",
                                                        layout={"width": "405px"},
                                                        style={"font_weight": "bold"})    
    
    settings_dict.update({"variable": "biomass_density"})

    def update_settings_dict(change):
        settings_dict.update({
            "variable": (
                "biomass_density" if empirical_variogram_dropdown.value == "Biomass (biomass density)" 
                else "number_density"
            ),
        })

    # Initialize the value dictionary
    general_settings = {
        "n_lags": n_lags_entry.value,
        "lag_resolution": lag_resolution_entry.value,
        "azimuth_range": azimuth_range_entry.value,
        "variable": empirical_variogram_dropdown.value,
    }

    def compute_empirical_variogram(transect_data, general_settings, settings_dict):
        # Attach the update function to the dropdown value change event
        empirical_variogram_dropdown.observe(update_settings_dict, names='value')
        # Compute the distance lags
        distance_lags = np.arange(1, general_settings["n_lags"]) * general_settings["lag_resolution"]
        # ---- Compute the maximum range
        max_range = general_settings["n_lags"] * general_settings["lag_resolution"]

        # Update the general settings for computing the empirical variogram
        general_settings.update({"force_lag_zero": True, "distance_lags": distance_lags, 
                                "range": max_range})
        
        # Compute the empirical variogram
        lags, gamma_h, lag_counts, _ = empirical_variogram(
            transect_data, general_settings, settings_dict
        )

        # Return the outputs
        return lags, gamma_h, lag_counts, general_settings
    
    empirical_params = {}
    # # Button-click
    def click_empirical_variogram_button(button, empirical_params):
        # Show the loading indicator
        loading_label.value = "Computing the empirical variogram, please wait..."
        # Compute the empirical variogram
        
        lags, gamma_h, lag_counts, _ = compute_empirical_variogram(transect_data, general_settings, settings_dict)
        # empirical_params = {}
        empirical_params["lags"] = lags
        empirical_params["gamma_h"] = gamma_h
        empirical_params["lag_counts"] = lag_counts
        # Clear the previous plot
        plot_output_widget.clear_output()

        # Plot the new empirical variogram
        with plot_output_widget:
            fig, ax = plot_empirical_variogram(lags, gamma_h, lag_counts)
            plt.show(fig)

        # Hide the loading indicator
        loading_label.value = ""

        # return empirical_params

    # Attach the button click event to the handler function
    compute_empirical_variogram_button.on_click(lambda button: click_empirical_variogram_button(button, empirical_params))


    # compute_empirical_variogram_button.on_click(click_empirical_variogram_button)

    # Create helper function that updates the current entry values
    def update_general_settings(change):
        # ---- Number of lags
        general_settings["n_lags"] = n_lags_entry.value
        # ---- Lag (increment) resolution
        general_settings["lag_resolution"] = lag_resolution_entry.value
        # ---- Azimuth range 
        general_settings["azimuth_range"] = azimuth_range_entry.value

    def register_observers(widgets, handler, names='value'):
        for widget in widgets:
            widget.observe(handler, names=names)

    # Register the widget observers
    register_observers([n_lags_entry, lag_resolution_entry, azimuth_range_entry], 
                       update_general_settings)
    
    # Vertically concatenate/bundle widgets
    widget_box = widgets.VBox(
        [n_lags_entry, lag_resolution_entry, azimuth_range_entry, empirical_variogram_dropdown,
        compute_empirical_variogram_button]
    )

    # Return the widget and settings dictionary
    return widget_box, general_settings, empirical_params

def compute_theoretical_variogram(lags, parameters):
    # Get the current input values
    arg_dict = {
        key: value.value if isinstance(value, (widgets.FloatText, widgets.Checkbox)) else value
        for key, value in parameters.items()
    }

    # pass into variogram
    
    distance_lags = lags
    return variogram(distance_lags, arg_dict)

def plot_theoretical_variogram(fig, ax, parameters):
    lags = empirical_params["lags"]
    theoretical_variogram = compute_theoretical_variogram(lags, parameters)
    
    ax.plot(lags, theoretical_variogram, linewidth = 3.0, c = "black")
    # plt.plot(lags, theoretical_variogram)
    fig.canvas.draw_idle()
        
def stitch_variogram_accordion(settings_widget, variogram_widget, optimization_widget):

    # Initialize the accordion tabs
    variogram_accordion = widgets.Accordion(layout=dict(width="450px"))

    # Populate the accordion children
    variogram_accordion.children = [settings_widget, variogram_widget, optimization_widget]

    # Default first tab
    variogram_accordion.selected_index = 0

    # Set the accordion tab titles
    variogram_accordion.set_title(0, "Empirical variogram", )
    variogram_accordion.set_title(1, "Theoretical variogram", )
    variogram_accordion.set_title(2, "Optimize variogram parameters", )

    # Return the accordion
    return variogram_accordion

def get_variogram_defaults(variogram_parameters, argument):
    DEFAULT_PARAMETERS = {
        "nugget": 0.0,
        "sill": 1.0,
        "hole_effect_range": 0.0,
        "decay_power": 1.5,
        "enhance_semivariance": False,
    }
    # ---- get variogram config
    if argument in variogram_parameters.keys():
        return variogram_parameters[argument]
    else:
        return DEFAULT_PARAMETERS[argument]


def theoretical_variogram_widgets(empirical_params):
    dropdown_variogram_model = widgets.Dropdown(
        options=list(VARIOGRAM_MODEL_PARAMETER_MAP.keys()),
        value="Bessel-exponential",
        description="Variogram model",
        disabled=False,
        style = {"description_width": "153px"}, 
        layout = {"width": "403px"}
    )

    # Create theoretical model computation button
    compute_theoretical_variogram_button = widgets.Button(
        description='Compute Theoretical Variogram',
        layout={"width": "405px"},
        style={'font_weight': 'bold'}
    )

    # Dynamically populate the model parameter field(s)
    # ---- Initialize the value entry dictionary
    variogram_settings = {}
    # ---- Initialize the Output widget
    variogram_model_widgets = widgets.Output()

    def dynamic_parameters(change):
        with variogram_model_widgets:
            # ---- Clear previous values
            clear_output(wait=True)
            # ---- Get the dropdown selection
            model_selection = change["new"]
            # ---- Retrieve the model argument used for `variogram(...)`
            model_name = VARIOGRAM_MODEL_PARAMETER_MAP[model_selection]
            # ---- Get argument names for this particular function
            function_arguments, _ = get_variogram_arguments(model_name)
            # ---- Model arguments (convert to dictionary)
            args_dict = dict(function_arguments)
            # ---- Clear `variogram_settings`
            variogram_settings.clear()
            # ---- Clear existing widgets, if any
            variogram_model_widgets.clear_output()
            # ---- Add the model name to the dictionary
            variogram_settings["model"] = model_name
            # ---- Generate widgets for each parameter
            for param in args_dict.values():
                # ---- Get the parameter name
                parameter = param.name
                # ---- Search reference dictionary for widget parameterization
                if parameter != "distance_lags":
                    # ---- Get default parameterization
                    default_value = get_variogram_defaults(variogram_parameters, parameter)
                    # ---- Get the function argument name
                    arg_name = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["name"]
                    # ---- Get widget-type
                    widget_type = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["widget"]
                    # ---- Get step size
                    step_size = THEORETICAL_VARIOGRAM_PARAMETER_MAP[parameter]["step"]
                    # ---- Create the appropriate widget
                    if widget_type == "checkbox":
                        checkbox = widgets.Checkbox(value=default_value, description=arg_name, 
                                                    style={"description_width": "initial"})
                        # ---- Add checkbox entry to `variogram_settings`
                        variogram_settings[parameter] = checkbox
                        # ---- Display the result
                        display(checkbox)
                    elif widget_type == "entry": 
                        entry = widgets.FloatText(value=default_value, description=arg_name, 
                                                style=style, layout=layout, step=step_size)
                        # ---- Add floattext entry to `variogram_settings`
                        variogram_settings[parameter] = entry
                        # ---- Display the result
                        display(entry)            

    # Attach the observer function to the dropdown menu
    dropdown_variogram_model.observe(dynamic_parameters, names="value")

    # Register the updated parameters
    dynamic_parameters({"new": dropdown_variogram_model.value})    

    def on_compute_button_clicked(button):
        # parameters = variogram_widgets
        parameters = variogram_settings
        with plot_output_widget: 
            # out.clear_output(wait=True)  # Clear previous plot
            clear_output(wait=True)
            patch_initial = Patch(color="black", linewidth=1, label="Initial theoretical variogram")
            fig, ax = plot_empirical_variogram(**empirical_params, patch=patch_initial)
            plot_theoretical_variogram(fig, ax, parameters)
            fig.canvas.toolbar_visible = False
            fig.canvas.header_visible = False
            fig.canvas.footer_visible = False
            fig.canvas.resizable = False
            plt.show(fig)

    # Update plot
    compute_theoretical_variogram_button.on_click(on_compute_button_clicked)

    widget_box = widgets.VBox([
        dropdown_variogram_model,
        variogram_model_widgets,
        compute_theoretical_variogram_button
    ])

    return widget_box, variogram_settings

def optimize_variogram_widgets(variogram_settings, general_settings, empirical_params):
    
    # Create button
    optimize_variogram_button = widgets.Button(
        description='Optimize variogram parameters',
        layout={"width": "405px"},
        style={'font_weight': 'bold'}
    )
    
    # Create label widgets to display results
    best_fit_variogram_label = widgets.Label()
    # initial_fit_label = widgets.Label()
    # optimized_fit_label = widgets.Label()

    # Initialize dictionary for tracking argument values
    optimization_args = {}
    float_widgets = {}
    checkbox_widgets = {}

    # Create a container for the widgets
    widget_entries = []

    # Define layout and style
    opt_layout = {"width": "400px"}
    opt_style = {"description_width": "210px"}

    # # Function to update values dictionary
    def update_values(change):
        widget = change['owner']
        description = widget.description
        for desc, config in OPTIMIZATION_PARAMETER_MAP.items():
            if desc == description:
                widget_type = config["widget"]
                name = config["name"]

                if widget_type == "entry_int" or widget_type == "entry_float":
                    optimization_args[name] = widget.value
                elif widget_type == "entry_str":
                    optimization_args[name] = widget.value
                elif widget_type == "entry_hybrid":
                    checkbox_value = checkbox_widgets[description].value
                    if checkbox_value:
                        optimization_args[name] = "jacobian"
                    else:
                        optimization_args[name] = float_widgets[description].value

    # Create widgets based on OPTIMIZATION_PARAMETER_MAP
    for description, config in OPTIMIZATION_PARAMETER_MAP.items():
        widget_type = config["widget"]
        initial_value = config["value"]
        step = config["step"]
        name = config["name"]

        if widget_type == "entry_int":
            widget = widgets.IntText(value=initial_value, description=description, step=step, 
                                        layout=opt_layout, style=opt_style)
        elif widget_type == "entry_float":
            widget = widgets.FloatText(value=initial_value, description=description, step=step, 
                                        layout=opt_layout, style=opt_style)
        elif widget_type == "entry_str":
            widget = widgets.Dropdown(options=step, value=initial_value, description=description, 
                                        layout=opt_layout, style=opt_style)
        elif widget_type == "entry_hybrid":
            float_widget = widgets.FloatText(value=config["value_float"], description=description, layout=opt_layout, 
                                                style=opt_style, step=step, disabled=True)
            checkbox_widget = (
                widgets.Checkbox(description="Use Jacobian for characteristic x-scaling", 
                                    value=True, 
                                    layout=opt_layout, 
                                    style=opt_style)
            )

            # Function to toggle between float widget and disabled state
            def toggle_float_widget(change):
                if change.new:
                    optimization_args["x_scale"] = "jacobian"
                    float_widget.disabled = True
                else:
                    float_widget.disabled = False
                    optimization_args["x_scale"] = float(float_widget.value) 
            
            checkbox_widget.observe(toggle_float_widget, 'value')
            
            # Initial state based on default value
            if initial_value == "jacobian":
                float_widget.disabled = True
            
            widget = widgets.VBox([checkbox_widget, float_widget])

        # Attach observer to each widget to update values dictionary
        widget.observe(update_values, names='value')
        optimization_args[name] = initial_value 
        widget_entries.append(widget)
        
    def compute_optimal_variogram(best_fit_parameters, lags, model_def):
        
        # Add model name to `best_fit_parameters`
        # ---- Create copy
        best_fit_parameters_copy = best_fit_parameters.copy()
        # ---- Append model
        best_fit_parameters_copy.update({"model": model_def})
        
        # Compute variogram
        return variogram(lags, best_fit_parameters_copy)
        
    def plot_optimal_variogram(fig, ax, best_fit_variogram, empirical_params, model):
        
        # Get lags
        lags = empirical_params["lags"]
        
        # Compute the optimal variogram
        optimal_variogram = compute_optimal_variogram(best_fit_variogram, lags, model)
        
        ax.plot(lags, optimal_variogram, linewidth=3.0, c="red", label="Optimal variogram")
        fig.canvas.draw_idle()     
        
    def on_optimize_variogram_button_clicked(button, variogram_settings, general_settings):
        # Convert the widget dictionary into a normal dictionary
        parameters_dict = {
            key: value.value if isinstance(value, (widgets.FloatText, widgets.Checkbox)) else value
            for key, value in variogram_settings.items()
        }

        # Extract the model name (this is stored for later)
        model = parameters_dict["model"]
        # ---- Remove from the parameters dictionary
        del parameters_dict["model"]

        # Get the fit parameters
        fit_parameters = list(parameters_dict.keys())

        # Create a list of lower bound tuples
        # ---- Repeat zero's
        lower_bound_values = np.zeros(len(fit_parameters))
        # ---- Convert to tuples list
        lower_bounds = [w for w in zip(fit_parameters, lower_bound_values) if None not in w]

        # Create a tuples list for the upper bounds
        # ---- Repeat INF
        upper_bound_values = np.repeat(np.inf, len(fit_parameters))
        # ---- Convert to tuples list
        upper_bounds = [w for w in zip(fit_parameters, upper_bound_values) if None not in w]

        # Create a tuples list for the initial values
        initial_values = list(parameters_dict.items())

        # Create a refreshed parameter dictionary that includes both the model name and range
        # ---- Create copy
        variogram_parameters = parameters_dict.copy()
        # ---- Update the dictionary
        variogram_parameters.update({"model": model, "range": general_settings["range"]})

        # Create the correctly formatted optimization settings
        optimization_settings = create_optimization_options(
            fit_parameters,
            variogram_parameters,
            initial_values=initial_values,
            lower_bounds=lower_bounds,
            upper_bounds=upper_bounds,
            model=variogram_parameters["model"],
            **optimization_args,
        )

        # Validate all variogram-related inputs
        validate_variogram_parameters(variogram_parameters, fit_parameters, optimization_settings)

        # Find the best-fit parameters
        best_fit_variogram, initial_fit, optimized_fit = optimize_variogram(
            empirical_params["lag_counts"], empirical_params["lags"], empirical_params["gamma_h"], 
            variogram_parameters, optimization_settings
        )
        
        # Update plot 
        initial_parameters = variogram_settings
        with plot_output_widget:
            clear_output(wait=True)
            patch_initial = Patch(color="black", linewidth=1, label="Initial theoretical variogram")
            patch_optim = Patch(color="red", linewidth=1, label="Optimized variogram")
            fig, ax = plot_empirical_variogram(**empirical_params, patch=[patch_initial, patch_optim])
            plot_theoretical_variogram(fig, ax, initial_parameters)
            plot_optimal_variogram(fig, ax, best_fit_variogram, empirical_params, model)
            fig.canvas.toolbar_visible = False
            fig.canvas.header_visible = False
            fig.canvas.footer_visible = False
            fig.canvas.resizable = False
            plt.show(fig)    
            
        # Store the values 
        # on_optimize_variogram_button_clicked.best_fit_variogram = best_fit_variogram
        # on_optimize_variogram_button_clicked.initial_fit = initial_fit
        # on_optimize_variogram_button_clicked.optimized_fit = optimized_fit
        # Update the label widgets with the results
        best_fit_variogram_str = "\n".join(
            f"{format(value, THEORETICAL_VARIOGRAM_PARAMETER_MAP.get(key, ''))}" 
            for key, value in best_fit_variogram.items()
        )
        best_fit_variogram_label.value = f"Best fit variogram: {best_fit_variogram_str}"        
        # initial_fit_label.value = f"Initial Fit: {initial_fit}"
        # optimized_fit_label.value = f"Optimized Fit: {optimized_fit}"
        
    optimize_variogram_button.on_click(lambda b: on_optimize_variogram_button_clicked(b, variogram_settings, general_settings))

    # Display the widgets
    entry_widgets = widgets.VBox(widget_entries)
    return widgets.VBox([entry_widgets, optimize_variogram_button, best_fit_variogram_label]), optimization_args

In [17]:
%matplotlib widget
plot_output_widget = widgets.Output(layout=widgets.Layout(height="550px"))
loading_label = widgets.Label(value="")
tab_empirical_variogram, general_settings, empirical_params = empirical_variogram_widget()
tab_contents_variogram, variogram_settings = theoretical_variogram_widgets(empirical_params)
tab_contents_optimization, optimization_settings = optimize_variogram_widgets(variogram_settings, general_settings, empirical_params)

full_accordion = stitch_variogram_accordion(tab_empirical_variogram, 
                                            tab_contents_variogram, 
                                            tab_contents_optimization)

widgets.HBox([full_accordion, widgets.VBox([loading_label, plot_output_widget])])

TypeError: format() argument 2 must be str, not dict

In [15]:
best_fit_variogram_str = "\n".join(
    f"{key}: {format(value, THEORETICAL_VARIOGRAM_PARAMETER_MAP.get(key, ''))}" 
    for key, value in best_fit_variogram.items()
)
f"Best fit variogram: {best_fit_variogram_str}"     

NameError: name 'best_fit_variogram' is not defined